## 完成ver.

In [1]:
#ライブラリのインポート
import geopandas as gpd
from osgeo import osr, ogr, gdal
from pyproj import CRS
from shapely.geometry import Point
import copy
from geopandas.tools import sjoin_nearest
import pandas as pd

In [2]:
#道路データとマンホールデータのパス
road_path = "C:/Users/jin12/Desktop/pre_intern/14100 (1).gpkg"
point_path = "C:/Users/jin12/Desktop/pre_intern/drive-download-20230716T102550Z-001/manfp_marge.shp"

In [3]:
#道路データとマンホールデータをデータフレームで読み込む
road_data = gpd.read_file(road_path)
point_data = gpd.read_file(point_path)

In [4]:
#元データのコピーを作る
point_data_test = copy.deepcopy(point_data)
road_data_test = copy.deepcopy(road_data)

In [5]:
#座標測地系の確認
print(point_data_test.crs)

None


In [6]:
print(road_data_test.crs)

GEOGCS["JGD2011",DATUM["Japanese_Geodetic_Datum_2011",SPHEROID["GRS 1980",6378137,298.257222101],AUTHORITY["EPSG","1128"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]


In [7]:
# インデックスをリセットして新しいカラムを作成
point_data_test = point_data_test.reset_index()  # インデックスをリセット
point_data_test['point_column'] = point_data_test.index  # 新しいカラムにインデックスを設定

# 結果を表示
point_data_test

,index,ANGLE,FC,FUKASA,FUTAJIBAN1,FUTAJIBAN2,FUTAJIBAN3,FUTAJIBAN4,FUTAJIBAN5,FUTAKEY1,...,PUMP,SISETU,SUISOKU,TAKASA,TYPE,YOTO,ZAISITU,ZANCHI,geometry,point_column
0,0,137.001700,12245,0.0,82.12,0.00,0.00,0.0,0.0,1,...,0,0,0,0.0,2,0,14,0,POINT Z (-27800.381 -45295.275 0.000),0
1,1,93.124200,12144,0.0,0.00,0.00,0.00,0.0,0.0,None,...,0,0,0,0.0,1,0,14,0,POINT Z (-27800.582 -45293.185 0.000),1
2,2,114.999200,12244,0.0,0.00,0.00,0.00,0.0,0.0,None,...,0,0,0,0.0,1,0,14,0,POINT Z (-27818.468 -45177.203 0.000),2
3,3,62.999300,12144,0.0,0.00,0.00,0.00,0.0,0.0,None,...,0,0,0,0.0,1,0,14,0,POINT Z (-27817.067 -45181.984 0.000),3
4,4,122.000500,12244,0.0,0.00,0.00,0.00,0.0,0.0,None,...,0,0,0,0.0,1,0,14,0,POINT Z (-27818.815 -45190.237 0.000),4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615188,615188,203.732706,12344,0.0,38.69,2.12,3.56,0.0,0.0,1,...,9,0,0,0.0,1,31,99,6,POINT Z (-16634.329 -50026.464 0.000),615188
615189,615189,199.249267,12344,0.0,16.86,2.12,3.56,0.0,0.0,1,...,9,0,0,0.0,40,31,99,6,POINT Z (-18186.962 -52823.785 0.000),615189
615190,615190,124.629046,12344,0.0,39.21,2.12,3.56,0.0,0.0,1,...,9,0,0,0.0,4,31,99,6,POINT Z (-24516.360 -51980.284 0.000),615190
615191,615191,77.305967,12344,0.0,3.97,2.12,3.56,0.0,0.0,1,...,9,0,0,0.0,1,31,99,6,POINT Z (-19588.746 -52532.906 0.000),615191


In [8]:
#マンホールデータのCRSを4326にする
point_data_test = point_data_test.set_crs(epsg=6677)
point_data_test = point_data_test.to_crs(epsg=4326)
point_data_test.head()

,index,ANGLE,FC,FUKASA,FUTAJIBAN1,FUTAJIBAN2,FUTAJIBAN3,FUTAJIBAN4,FUTAJIBAN5,FUTAKEY1,...,PUMP,SISETU,SUISOKU,TAKASA,TYPE,YOTO,ZAISITU,ZANCHI,geometry,point_column
0,0,137.0017,12245,0.0,82.12,0.0,0.0,0.0,0.0,1,...,0,0,0,0.0,2,0,14,0,POINT (139.52655 35.59134),0
1,1,93.1242,12144,0.0,0.00,0.0,0.0,0.0,0.0,None,...,0,0,0,0.0,1,0,14,0,POINT (139.52654 35.59136),1
2,2,114.9992,12244,0.0,0.00,0.0,0.0,0.0,0.0,None,...,0,0,0,0.0,1,0,14,0,POINT (139.52634 35.59240),2
3,3,62.9993,12144,0.0,0.00,0.0,0.0,0.0,0.0,None,...,0,0,0,0.0,1,0,14,0,POINT (139.52636 35.59236),3
4,4,122.0005,12244,0.0,0.00,0.0,0.0,0.0,0.0,None,...,0,0,0,0.0,1,0,14,0,POINT (139.52634 35.59228),4


In [9]:
# インデックスをリセットして新しいカラムを作成
road_data_test['road_column'] = road_data_test.index  # 新しいカラムにインデックスを設定

# 結果を表示
road_data_test

,WKT,rID,lfSpanFr,lfSpanTo,tmpFlg,orgGILvl,ftCode,admCode,devDate,type,...,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry,road_column
0,"MULTILINESTRING ((139.670389056 35.470258527,1...",50280-12768-i-111,2022/06/06,,0,25000,2711,,,庭園路,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67039 35.47026, 139.6683...",0
1,"MULTILINESTRING ((139.669480483 35.472380181,1...",50280-12768-i-118,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6692...",1
2,"MULTILINESTRING ((139.669480483 35.472380181,1...",50280-12768-i-117,2022/06/06,,0,2500,2701,,2017/10/24,通常部,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6701...",2
3,"MULTILINESTRING ((139.668721043 35.467976544,1...",50280-12768-i-115,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66872 35.46798, 139.6710...",3
4,"MULTILINESTRING ((139.670150145 35.470816418,1...",50280-12768-i-119,2022/06/06,,0,2500,2701,,2017/10/24,通常部,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67015 35.47082, 139.6703...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235606,"MULTILINESTRING ((139.568341134 35.568819865,1...",50244-12804-i-8190,2022/06/06,,0,2500,2701,,,通常部,...,,,22,1696,580,139.566873,35.568595,0540-00,"MULTILINESTRING ((139.56834 35.56882, 139.5674...",235606
235607,"MULTILINESTRING ((139.568141861 35.568553389,1...",50244-12804-i-8187,2022/06/06,,0,2500,2701,,,通常部,...,,,22,1696,580,139.566873,35.568595,0540-00,"MULTILINESTRING ((139.56814 35.56855, 139.5676...",235607
235608,"MULTILINESTRING ((139.568141861 35.568553389,1...",50244-12804-i-8185,2022/06/06,,0,2500,2701,,,通常部,...,,,22,1696,580,139.566873,35.568595,0540-00,"MULTILINESTRING ((139.56814 35.56855, 139.5676...",235608
235609,"MULTILINESTRING ((139.567572167 35.570356611,1...",50244-12804-i-8181,2022/06/06,,0,2500,2701,,,通常部,...,,,22,1696,580,139.566873,35.568595,0540-00,"MULTILINESTRING ((139.56757 35.57036, 139.5684...",235609


In [10]:
#道路データのCRSを4326にする
road_data_test = road_data_test.to_crs(epsg=4326)
road_data_test.head()

,WKT,rID,lfSpanFr,lfSpanTo,tmpFlg,orgGILvl,ftCode,admCode,devDate,type,...,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry,road_column
0,"MULTILINESTRING ((139.670389056 35.470258527,1...",50280-12768-i-111,2022/06/06,,0,25000,2711,,,庭園路,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67039 35.47026, 139.6683...",0
1,"MULTILINESTRING ((139.669480483 35.472380181,1...",50280-12768-i-118,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6692...",1
2,"MULTILINESTRING ((139.669480483 35.472380181,1...",50280-12768-i-117,2022/06/06,,0,2500,2701,,2017/10/24,通常部,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6701...",2
3,"MULTILINESTRING ((139.668721043 35.467976544,1...",50280-12768-i-115,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66872 35.46798, 139.6710...",3
4,"MULTILINESTRING ((139.670150145 35.470816418,1...",50280-12768-i-119,2022/06/06,,0,2500,2701,,2017/10/24,通常部,...,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67015 35.47082, 139.6703...",4


In [11]:
# # GeoDataFrameをGeoJSONファイルに出力
# output_file = 'road_data.geojson'
# road_data_test.to_file(output_file, driver='GeoJSON')

In [55]:
#空間結合をする
result = gpd.sjoin_nearest(point_data_test, road_data_test, how="left",lsuffix='point',rsuffix='road')

c:\Users\jin12\anaconda3\envs\god\lib\site-packages\geopandas\array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [ ]:
# # GeoDataFrameをGeoJSONファイルに出力
# output_file = 'output.geojson'
# result.to_file(output_file, driver='GeoJSON')

## 確認作業

In [56]:
result_check = result[["point_column","road_column", "geometry"]]


In [57]:
# インデックスをリセットして新しいカラムを作成
result_check = result_check.reset_index()  # インデックスをリセット
result_check['result_column'] = result_check.index  # 新しいカラムにインデックスを設定

# 結果を表示
result_check

,index,point_column,road_column,geometry,result_column
0,0,0,219309,POINT (139.52655 35.59134),0
1,1,1,219309,POINT (139.52654 35.59136),1
2,2,2,219318,POINT (139.52634 35.59240),2
3,3,3,219318,POINT (139.52636 35.59236),3
4,4,4,219318,POINT (139.52634 35.59228),4
...,...,...,...,...,...
620030,615188,615188,100650,POINT (139.64986 35.54894),620030
620031,615189,615189,98234,POINT (139.63280 35.52370),620031
620032,615190,615190,220980,POINT (139.56299 35.53117),620032
620033,615191,615191,96781,POINT (139.61734 35.52630),620033


In [73]:
result_check[result_check["point_column"]==1507]

,index,point_column,road_column,geometry,result_column
1511,1507,1507,219270,POINT (139.54218 35.58831),1511
1512,1507,1507,219267,POINT (139.54218 35.58831),1512


In [58]:
# GeoDataFrameをGeoJSONファイルに出力
output_file = 'result_check.geojson'
result_check.to_file(output_file, driver='GeoJSON')

In [14]:
result_check.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 620035 entries, 0 to 615192
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   new_column  620035 non-null  int64   
 1   geometry    620035 non-null  geometry
dtypes: geometry(1), int64(1)
memory usage: 14.2 MB


In [68]:
# poi
result_check[result_check.duplicated(subset="point_column")].head(50)

,index,point_column,road_column,geometry,result_column
1511,1507,1507,219270,POINT (139.54218 35.58831),1511
1512,1507,1507,219267,POINT (139.54218 35.58831),1512


In [76]:
# 重複している値を抽出
duplicates = result_check[result_check.duplicated(subset="point_column", keep=False)]

# 重複している値を表示
print(duplicates)


         index  point_column  road_column                    geometry  \
1010      1010          1010       219488  POINT (139.53459 35.58931)   
1011      1010          1010       219481  POINT (139.53459 35.58931)   
1012      1010          1010       219482  POINT (139.53459 35.58931)   
1158      1156          1156       219501  POINT (139.53638 35.58868)   
1159      1156          1156       219503  POINT (139.53638 35.58868)   
...        ...           ...          ...                         ...   
618953  614113        614113       182576  POINT (139.56361 35.36323)   
619108  614268        614268       227608  POINT (139.54759 35.53108)   
619109  614268        614268       227258  POINT (139.54759 35.53108)   
619584  614743        614743        65879  POINT (139.62027 35.42600)   
619585  614743        614743        43360  POINT (139.62027 35.42600)   

        result_column  
1010             1010  
1011             1011  
1012             1012  
1158             1158  
115

In [15]:
# 重複している値を抽出
duplicates_road = road_data_test[road_data_test.duplicated(subset="rID", keep=False)]

# 重複している値を表示
duplicates_road.head(10)

,WKT,rID,lfSpanFr,lfSpanTo,tmpFlg,orgGILvl,ftCode,admCode,devDate,type,...,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry,road_column
324,"MULTILINESTRING ((139.667650556 35.493152583,1...",50280-12777-i-2877,2022/06/06,,0,25000,2701,,,通常部,...,,,36,1981,1360,139.665293,35.490894,0030-01,"MULTILINESTRING ((139.66765 35.49315, 139.6678...",324
583,"MULTILINESTRING ((139.667650556 35.493152583,1...",50280-12777-i-2877,2022/06/06,,0,25000,2701,,,通常部,...,,,43,3829,2406,139.669172,35.49454,0030-03,"MULTILINESTRING ((139.66765 35.49315, 139.6678...",583
950,"MULTILINESTRING ((139.66174954 35.494103142,13...",50277-12777-i-2557,2022/06/06,,0,25000,2701,,,通常部,...,,,31,2470,1302,139.663592,35.495097,0080-01,"MULTILINESTRING ((139.66175 35.49410, 139.6615...",950
969,"MULTILINESTRING ((139.663193444 35.497948222,1...",50277-12777-i-2500,2022/06/06,,0,25000,2701,,,通常部,...,,,31,2470,1302,139.663592,35.495097,0080-01,"MULTILINESTRING ((139.66319 35.49795, 139.6630...",969
1031,"MULTILINESTRING ((139.66174954 35.494103142,13...",50277-12777-i-2557,2022/06/06,,0,25000,2701,,,通常部,...,,,26,2054,1024,139.660953,35.495398,0080-02,"MULTILINESTRING ((139.66175 35.49410, 139.6615...",1031
1099,"MULTILINESTRING ((139.663193444 35.497948222,1...",50277-12777-i-2500,2022/06/06,,0,25000,2701,,,通常部,...,,,26,2054,1024,139.660953,35.495398,0080-02,"MULTILINESTRING ((139.66319 35.49795, 139.6630...",1099
1109,"MULTILINESTRING ((139.659747181 35.495683806,1...",50277-12777-i-2678,2022/06/06,,0,25000,2701,,,通常部,...,,,26,2054,1024,139.660953,35.495398,0080-02,"MULTILINESTRING ((139.65975 35.49568, 139.6597...",1109
1232,"MULTILINESTRING ((139.666874333 35.500167944,1...",50280-12780-i-2406,2022/06/06,,0,25000,2701,,,通常部,...,,,45,3892,1889,139.665154,35.501077,0080-03,"MULTILINESTRING ((139.66687 35.50017, 139.6667...",1232
1354,"MULTILINESTRING ((139.666874333 35.500167944,1...",50280-12780-i-2406,2022/06/06,,0,25000,2701,,,通常部,...,,,42,3435,1773,139.668189,35.499024,0080-04,"MULTILINESTRING ((139.66687 35.50017, 139.6667...",1354
1569,"MULTILINESTRING ((139.667331871 35.510367032,1...",50280-12783-i-4034,2022/06/06,,0,25000,2701,,,通常部,...,,,32,2560,1416,139.668877,35.511191,0140-01,"MULTILINESTRING ((139.66733 35.51037, 139.6682...",1569


In [18]:
duplicates_road_new = duplicates_road.drop(["road_column"], axis=1)
duplicates_road_new

,WKT,rID,lfSpanFr,lfSpanTo,tmpFlg,orgGILvl,ftCode,admCode,devDate,type,...,N_KEN,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry
324,"MULTILINESTRING ((139.667650556 35.493152583,1...",50280-12777-i-2877,2022/06/06,,0,25000,2701,,,通常部,...,,,,36,1981,1360,139.665293,35.490894,0030-01,"MULTILINESTRING ((139.66765 35.49315, 139.6678..."
583,"MULTILINESTRING ((139.667650556 35.493152583,1...",50280-12777-i-2877,2022/06/06,,0,25000,2701,,,通常部,...,,,,43,3829,2406,139.669172,35.49454,0030-03,"MULTILINESTRING ((139.66765 35.49315, 139.6678..."
950,"MULTILINESTRING ((139.66174954 35.494103142,13...",50277-12777-i-2557,2022/06/06,,0,25000,2701,,,通常部,...,,,,31,2470,1302,139.663592,35.495097,0080-01,"MULTILINESTRING ((139.66175 35.49410, 139.6615..."
969,"MULTILINESTRING ((139.663193444 35.497948222,1...",50277-12777-i-2500,2022/06/06,,0,25000,2701,,,通常部,...,,,,31,2470,1302,139.663592,35.495097,0080-01,"MULTILINESTRING ((139.66319 35.49795, 139.6630..."
1031,"MULTILINESTRING ((139.66174954 35.494103142,13...",50277-12777-i-2557,2022/06/06,,0,25000,2701,,,通常部,...,,,,26,2054,1024,139.660953,35.495398,0080-02,"MULTILINESTRING ((139.66175 35.49410, 139.6615..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235350,"MULTILINESTRING ((139.556403127 35.535433751,1...",50238-12792-i-3636,2022/06/06,,0,2500,2701,,,通常部,...,,,,44,2955,1136,139.554203,35.537279,0470-00,"MULTILINESTRING ((139.55640 35.53543, 139.5564..."
235356,"MULTILINESTRING ((139.553433847 35.533108621,1...",50238-12789-i-3463,2022/06/06,,0,2500,2701,,,通常部,...,,,,44,2955,1136,139.554203,35.537279,0470-00,"MULTILINESTRING ((139.55343 35.53311, 139.5536..."
235407,"MULTILINESTRING ((139.568377481 35.53644423,13...",50244-12792-i-1853,2022/06/06,,0,2500,2701,,,通常部,...,,,,28,1381,471,139.56963,35.535534,0520-00,"MULTILINESTRING ((139.56838 35.53644, 139.5689..."
235408,"MULTILINESTRING ((139.567854552 35.536448777,1...",50244-12792-i-1855,2022/06/06,,0,2500,2701,,,通常部,...,,,,28,1381,471,139.56963,35.535534,0520-00,"MULTILINESTRING ((139.56785 35.53645, 139.5677..."


In [19]:
# 重複している値を抽出
duplicates_road_new_check = duplicates_road_new[duplicates_road_new.duplicated(keep=False)]
duplicates_road_new_check

,WKT,rID,lfSpanFr,lfSpanTo,tmpFlg,orgGILvl,ftCode,admCode,devDate,type,...,N_KEN,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry


In [13]:
# 道路データの重複を出力
output_file = 'duplicates_road.geojson'
duplicates_road.to_file(output_file, driver='GeoJSON')

In [72]:
duplicates[duplicates["point_column"]==1507]

,index,point_column,road_column,geometry,result_column
1512,1507,1507,219267,POINT (139.54218 35.58831),1512


In [77]:
# GeoDataFrameをGeoJSONファイルに出力
output_file = 'duplicates.geojson'
duplicates.to_file(output_file, driver='GeoJSON')

In [ ]:
# GeoDataFrameをGeoJSONファイルに出力
output_file = 'result_check.geojson'
result_check.to_file(output_file, driver='GeoJSON')

In [78]:
point_data_test.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 615193 entries, 0 to 615192
Data columns (total 47 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   index         615193 non-null  int64   
 1   ANGLE         615193 non-null  float64 
 2   FC            615193 non-null  int64   
 3   FUKASA        615193 non-null  float64 
 4   FUTAJIBAN1    615193 non-null  float64 
 5   FUTAJIBAN2    615193 non-null  float64 
 6   FUTAJIBAN3    615193 non-null  float64 
 7   FUTAJIBAN4    615193 non-null  float64 
 8   FUTAJIBAN5    615193 non-null  float64 
 9   FUTAKEY1      133943 non-null  object  
 10  FUTAKEY2      8019 non-null    object  
 11  FUTAKEY3      4030 non-null    object  
 12  FUTAKEY4      46 non-null      object  
 13  FUTAKEY5      11 non-null      object  
 14  FUTASUU       615193 non-null  int64   
 15  G1KANRI       574383 non-null  object  
 16  G1KYOKA       1 non-null       object  
 17  G1KYOKAYMD    615193 

In [23]:
# pandasのoptionsを設定してカラム情報を全て表示
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

# データフレームの情報を表示
print(result.info(verbose=True))

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 620035 entries, 0 to 615192
Data columns (total 101 columns):
 #    Column      Dtype   
---   ------      -----   
 0    ANGLE       float64 
 1    FC          int64   
 2    FUKASA      float64 
 3    FUTAJIBAN1  float64 
 4    FUTAJIBAN2  float64 
 5    FUTAJIBAN3  float64 
 6    FUTAJIBAN4  float64 
 7    FUTAJIBAN5  float64 
 8    FUTAKEY1    object  
 9    FUTAKEY2    object  
 10   FUTAKEY3    object  
 11   FUTAKEY4    object  
 12   FUTAKEY5    object  
 13   FUTASUU     int64   
 14   G1KANRI     object  
 15   G1KYOKA     object  
 16   G1KYOKAYMD  int64   
 17   G2KANRI     object  
 18   G2KYOKA     object  
 19   G2KYOKAYMD  int64   
 20   GAIKEIH     float64 
 21   GAIKEIV     float64 
 22   HAIJYO      int64   
 23   HIKITUGI    object  
 24   JIBANTAKA   float64 
 25   JINSIN      float64 
 26   JINX        int64   
 27   JINY        int64   
 28   KAIDAN      int64   
 29   KANEDA      int64   
 30   KENSETUYMD

In [ ]:
# 重複している値を抽出
duplicates = result[result["geometry"].duplicated()]

# 重複している値を表示
print(duplicates)

In [ ]:
point_data_test.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 615193 entries, 0 to 615192
Data columns (total 45 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ANGLE       615193 non-null  float64 
 1   FC          615193 non-null  int64   
 2   FUKASA      615193 non-null  float64 
 3   FUTAJIBAN1  615193 non-null  float64 
 4   FUTAJIBAN2  615193 non-null  float64 
 5   FUTAJIBAN3  615193 non-null  float64 
 6   FUTAJIBAN4  615193 non-null  float64 
 7   FUTAJIBAN5  615193 non-null  float64 
 8   FUTAKEY1    133943 non-null  object  
 9   FUTAKEY2    8019 non-null    object  
 10  FUTAKEY3    4030 non-null    object  
 11  FUTAKEY4    46 non-null      object  
 12  FUTAKEY5    11 non-null      object  
 13  FUTASUU     615193 non-null  int64   
 14  G1KANRI     574383 non-null  object  
 15  G1KYOKA     1 non-null       object  
 16  G1KYOKAYMD  615193 non-null  int64   
 17  G2KANRI     574387 non-null  object  
 18  G2KYOKA     3948

In [ ]:
road_data_test.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 235611 entries, 0 to 235610
Data columns (total 56 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   WKT         235611 non-null  object  
 1   rID         235611 non-null  object  
 2   lfSpanFr    235611 non-null  object  
 3   lfSpanTo    235611 non-null  object  
 4   tmpFlg      235611 non-null  object  
 5   orgGILvl    235611 non-null  object  
 6   ftCode      235611 non-null  object  
 7   admCode     235611 non-null  object  
 8   devDate     235611 non-null  object  
 9   type        235611 non-null  object  
 10  rdCtg       235611 non-null  object  
 11  state       235611 non-null  object  
 12  lvOrder     235611 non-null  object  
 13  name        235611 non-null  object  
 14  admOfcRd    235611 non-null  object  
 15  rnkWidth    235611 non-null  object  
 16  Width       235611 non-null  object  
 17  sectID      235611 non-null  object  
 18  tollSect    2356

In [ ]:
result.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 620035 entries, 0 to 615192
Columns: 101 entries, ANGLE to KCODE1
dtypes: float64(14), geometry(1), int64(20), object(66)
memory usage: 482.5+ MB


In [ ]:
result.isnull().sum()

ANGLE              0
FC                 0
FUKASA             0
FUTAJIBAN1         0
FUTAJIBAN2         0
FUTAJIBAN3         0
FUTAJIBAN4         0
FUTAJIBAN5         0
FUTAKEY1      484887
FUTAKEY2      611961
FUTAKEY3      615983
FUTAKEY4      619988
FUTAKEY5      620023
FUTASUU            0
G1KANRI        41215
G1KYOKA       620034
G1KYOKAYMD         0
G2KANRI        41211
G2KYOKA       222038
G2KYOKAYMD         0
GAIKEIH            0
GAIKEIV            0
HAIJYO             0
HIKITUGI      103788
JIBANTAKA          0
JINSIN             0
JINX               0
JINY               0
KAIDAN             0
KANEDA             0
KENSETUYMD         0
KEY                0
KITEN              0
KUMITATE           0
NAIKEIH            0
NAIKEIV            0
PUMP               0
SISETU             0
SUISOKU            0
TAKASA             0
TYPE               0
YOTO               0
ZAISITU            0
ZANCHI             0
geometry           0
index_road         0
WKT                0
rID          

## demo

In [ ]:
#ライブラリのインポート
import geopandas as gpd
from osgeo import osr, ogr, gdal
from pyproj import CRS
from shapely.geometry import Point
import copy
from geopandas.tools import sjoin_nearest


In [ ]:
#道路データとマンホールデータのパス
road_path = "C:/Users/jin12/Desktop/pre_intern/14100 (1).gpkg"
point_path = "C:/Users/jin12/Desktop/pre_intern/drive-download-20230716T102550Z-001/manfp_marge.shp"

In [ ]:
#道路データとマンホールデータをデータフレームで読み込む
road_data = gpd.read_file(road_path)
point_data = gpd.read_file(point_path)

In [ ]:
road_data.head()

,WKT,rID,lfSpanFr,lfSpanTo,tmpFlg,orgGILvl,ftCode,admCode,devDate,type,...,N_KEN,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry
0,"MULTILINESTRING ((139.670389056 35.470258527,1...",50280-12768-i-111,2022/06/06,,0,25000,2711,,,庭園路,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67039 35.47026, 139.6683..."
1,"MULTILINESTRING ((139.669480483 35.472380181,1...",50280-12768-i-118,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6692..."
2,"MULTILINESTRING ((139.669480483 35.472380181,1...",50280-12768-i-117,2022/06/06,,0,2500,2701,,2017/10/24,通常部,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6701..."
3,"MULTILINESTRING ((139.668721043 35.467976544,1...",50280-12768-i-115,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66872 35.46798, 139.6710..."
4,"MULTILINESTRING ((139.670150145 35.470816418,1...",50280-12768-i-119,2022/06/06,,0,2500,2701,,2017/10/24,通常部,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67015 35.47082, 139.6703..."


In [ ]:
road_data_test = road_data.drop('WKT',axis=1)

In [ ]:
road_data_test.head()

,rID,lfSpanFr,lfSpanTo,tmpFlg,orgGILvl,ftCode,admCode,devDate,type,rdCtg,...,N_KEN,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry
0,50280-12768-i-111,2022/06/06,,0,25000,2711,,,庭園路,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67039 35.47026, 139.6683..."
1,50280-12768-i-118,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6692..."
2,50280-12768-i-117,2022/06/06,,0,2500,2701,,2017/10/24,通常部,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6701..."
3,50280-12768-i-115,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66872 35.46798, 139.6710..."
4,50280-12768-i-119,2022/06/06,,0,2500,2701,,2017/10/24,通常部,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67015 35.47082, 139.6703..."


In [ ]:
road_data_test.crs

<Geographic 2D CRS: GEOGCS["JGD2011",DATUM["Japanese_Geodetic_Datum_20 ...>
Name: JGD2011
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: Japanese Geodetic Datum 2011
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
point_data.head()

,ANGLE,FC,FUKASA,FUTAJIBAN1,FUTAJIBAN2,FUTAJIBAN3,FUTAJIBAN4,FUTAJIBAN5,FUTAKEY1,FUTAKEY2,...,NAIKEIV,PUMP,SISETU,SUISOKU,TAKASA,TYPE,YOTO,ZAISITU,ZANCHI,geometry
0,137.0017,12245,0.0,82.12,0.0,0.0,0.0,0.0,1,None,...,0.0,0,0,0,0.0,2,0,14,0,POINT Z (-27800.381 -45295.275 0.000)
1,93.1242,12144,0.0,0.00,0.0,0.0,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT Z (-27800.582 -45293.185 0.000)
2,114.9992,12244,0.0,0.00,0.0,0.0,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT Z (-27818.468 -45177.203 0.000)
3,62.9993,12144,0.0,0.00,0.0,0.0,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT Z (-27817.067 -45181.984 0.000)
4,122.0005,12244,0.0,0.00,0.0,0.0,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT Z (-27818.815 -45190.237 0.000)


In [ ]:
point_data.crs

## memo

In [ ]:
#ライブラリのインポート
import geopandas as gpd

In [ ]:
#道路データとマンホールデータをデータフレームで読み込む
road_data = gpd.read_file(road_path)
point_data = gpd.read_file(point_path)

In [ ]:
#道路データの情報を確認
road_data.head()

,WKT,rID,lfSpanFr,lfSpanTo,tmpFlg,orgGILvl,ftCode,admCode,devDate,type,...,N_KEN,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry
0,"MULTILINESTRING ((139.670389056 35.470258527,1...",50280-12768-i-111,2022/06/06,,0,25000,2711,,,庭園路,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67039 35.47026, 139.6683..."
1,"MULTILINESTRING ((139.669480483 35.472380181,1...",50280-12768-i-118,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6692..."
2,"MULTILINESTRING ((139.669480483 35.472380181,1...",50280-12768-i-117,2022/06/06,,0,2500,2701,,2017/10/24,通常部,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6701..."
3,"MULTILINESTRING ((139.668721043 35.467976544,1...",50280-12768-i-115,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66872 35.46798, 139.6710..."
4,"MULTILINESTRING ((139.670150145 35.470816418,1...",50280-12768-i-119,2022/06/06,,0,2500,2701,,2017/10/24,通常部,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67015 35.47082, 139.6703..."


In [ ]:
road_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 235611 entries, 0 to 235610
Data columns (total 56 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   WKT         235611 non-null  object  
 1   rID         235611 non-null  object  
 2   lfSpanFr    235611 non-null  object  
 3   lfSpanTo    235611 non-null  object  
 4   tmpFlg      235611 non-null  object  
 5   orgGILvl    235611 non-null  object  
 6   ftCode      235611 non-null  object  
 7   admCode     235611 non-null  object  
 8   devDate     235611 non-null  object  
 9   type        235611 non-null  object  
 10  rdCtg       235611 non-null  object  
 11  state       235611 non-null  object  
 12  lvOrder     235611 non-null  object  
 13  name        235611 non-null  object  
 14  admOfcRd    235611 non-null  object  
 15  rnkWidth    235611 non-null  object  
 16  Width       235611 non-null  object  
 17  sectID      235611 non-null  object  
 18  tollSect    2356

In [ ]:
print(road_data.crs.to_epsg())

None


In [ ]:
#マンホールデータの情報を確認
point_data.head()

,ANGLE,FC,FUKASA,FUTAJIBAN1,FUTAJIBAN2,FUTAJIBAN3,FUTAJIBAN4,FUTAJIBAN5,FUTAKEY1,FUTAKEY2,...,SISETU,SUISOKU,TAKASA,TYPE,YOTO,ZAISITU,ZANCHI,geometry,Longitude,Latitude
0,137.0017,12245,0.0,82.12,0.0,0.0,0.0,0.0,1,None,...,0,0,0.0,2,0,14,0,POINT (36.000 139.835),36.000324,139.834884
1,93.1242,12144,0.0,0.00,0.0,0.0,0.0,0.0,None,None,...,0,0,0.0,1,0,14,0,POINT (36.000 139.835),36.000324,139.834884
2,114.9992,12244,0.0,0.00,0.0,0.0,0.0,0.0,None,None,...,0,0,0.0,1,0,14,0,POINT (36.000 139.835),36.000324,139.834884
3,62.9993,12144,0.0,0.00,0.0,0.0,0.0,0.0,None,None,...,0,0,0.0,1,0,14,0,POINT (36.000 139.835),36.000324,139.834884
4,122.0005,12244,0.0,0.00,0.0,0.0,0.0,0.0,None,None,...,0,0,0.0,1,0,14,0,POINT (36.000 139.835),36.000324,139.834884


In [ ]:
point_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 615193 entries, 0 to 615192
Data columns (total 45 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ANGLE       615193 non-null  float64 
 1   FC          615193 non-null  int64   
 2   FUKASA      615193 non-null  float64 
 3   FUTAJIBAN1  615193 non-null  float64 
 4   FUTAJIBAN2  615193 non-null  float64 
 5   FUTAJIBAN3  615193 non-null  float64 
 6   FUTAJIBAN4  615193 non-null  float64 
 7   FUTAJIBAN5  615193 non-null  float64 
 8   FUTAKEY1    133943 non-null  object  
 9   FUTAKEY2    8019 non-null    object  
 10  FUTAKEY3    4030 non-null    object  
 11  FUTAKEY4    46 non-null      object  
 12  FUTAKEY5    11 non-null      object  
 13  FUTASUU     615193 non-null  int64   
 14  G1KANRI     574383 non-null  object  
 15  G1KYOKA     1 non-null       object  
 16  G1KYOKAYMD  615193 non-null  int64   
 17  G2KANRI     574387 non-null  object  
 18  G2KYOKA     3948

In [ ]:
print(point_data.crs)

None


In [ ]:
point_data_test = copy.deepcopy(point_data)

In [ ]:
crs = CRS.from_epsg(6677)
# CRSを付与する
point_data_test.crs = crs
print(point_data_test.crs)

epsg:6677


In [ ]:
point_data_test = point_data_test.to_crs("EPSG:4326")

In [ ]:
point_data_test

,ANGLE,FC,FUKASA,FUTAJIBAN1,FUTAJIBAN2,FUTAJIBAN3,FUTAJIBAN4,FUTAJIBAN5,FUTAKEY1,FUTAKEY2,...,NAIKEIV,PUMP,SISETU,SUISOKU,TAKASA,TYPE,YOTO,ZAISITU,ZANCHI,geometry
0,137.001700,12245,0.0,82.12,0.00,0.00,0.0,0.0,1,None,...,0.0,0,0,0,0.0,2,0,14,0,POINT (139.52655 35.59134)
1,93.124200,12144,0.0,0.00,0.00,0.00,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT (139.52654 35.59136)
2,114.999200,12244,0.0,0.00,0.00,0.00,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT (139.52634 35.59240)
3,62.999300,12144,0.0,0.00,0.00,0.00,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT (139.52636 35.59236)
4,122.000500,12244,0.0,0.00,0.00,0.00,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT (139.52634 35.59228)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615188,203.732706,12344,0.0,38.69,2.12,3.56,0.0,0.0,1,2,...,0.0,9,0,0,0.0,1,31,99,6,POINT (139.64986 35.54894)
615189,199.249267,12344,0.0,16.86,2.12,3.56,0.0,0.0,1,2,...,0.6,9,0,0,0.0,40,31,99,6,POINT (139.63280 35.52370)
615190,124.629046,12344,0.0,39.21,2.12,3.56,0.0,0.0,1,2,...,0.0,9,0,0,0.0,4,31,99,6,POINT (139.56299 35.53117)
615191,77.305967,12344,0.0,3.97,2.12,3.56,0.0,0.0,1,2,...,0.0,9,0,0,0.0,1,31,99,6,POINT (139.61734 35.52630)


In [ ]:
print(road_data_test.crs)

epsg:4326


In [ ]:
# JGD2011（Japanese Geodetic Datum 2011）のEPSGコード（6647）を設定
jgd2011_epsg = 1128

# EPSG:4326（WGS 84）のEPSGコード（4326）を設定
wgs84_epsg = 4326

# JGD2011からEPSG:4326に座標変換
road_data_test.to_crs(epsg=wgs84_epsg, inplace=True)

# 変換後のラインデータを表示（可視化や保存などの処理に利用）
print(road_data_test)

                       rID    lfSpanFr lfSpanTo tmpFlg orgGILvl ftCode  \
0        50280-12768-i-111  2022/06/06               0    25000   2711   
1        50280-12768-i-118  2022/06/06               0     2500   2711   
2        50280-12768-i-117  2022/06/06               0     2500   2701   
3        50280-12768-i-115  2022/06/06               0     2500   2711   
4        50280-12768-i-119  2022/06/06               0     2500   2701   
...                    ...         ...      ...    ...      ...    ...   
235606  50244-12804-i-8190  2022/06/06               0     2500   2701   
235607  50244-12804-i-8187  2022/06/06               0     2500   2701   
235608  50244-12804-i-8185  2022/06/06               0     2500   2701   
235609  50244-12804-i-8181  2022/06/06               0     2500   2701   
235610  50244-12804-i-8180  2022/06/06               0     2500   2701   

       admCode     devDate type   rdCtg  ... N_KEN N_CITY KIGO_I KBSUM JINKO  \
0                           庭園路

In [ ]:
road_data_test

,rID,lfSpanFr,lfSpanTo,tmpFlg,orgGILvl,ftCode,admCode,devDate,type,rdCtg,...,N_KEN,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry
0,50280-12768-i-111,2022/06/06,,0,25000,2711,,,庭園路,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67039 35.47026, 139.6683..."
1,50280-12768-i-118,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6692..."
2,50280-12768-i-117,2022/06/06,,0,2500,2701,,2017/10/24,通常部,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66948 35.47238, 139.6701..."
3,50280-12768-i-115,2022/06/06,,0,2500,2711,,2017/10/24,庭園路,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.66872 35.46798, 139.6710..."
4,50280-12768-i-119,2022/06/06,,0,2500,2701,,2017/10/24,通常部,市区町村道等,...,,,,13,19,19,139.674467,35.479431,0010-00,"MULTILINESTRING ((139.67015 35.47082, 139.6703..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235606,50244-12804-i-8190,2022/06/06,,0,2500,2701,,,通常部,市区町村道等,...,,,,22,1696,580,139.566873,35.568595,0540-00,"MULTILINESTRING ((139.56834 35.56882, 139.5674..."
235607,50244-12804-i-8187,2022/06/06,,0,2500,2701,,,通常部,市区町村道等,...,,,,22,1696,580,139.566873,35.568595,0540-00,"MULTILINESTRING ((139.56814 35.56855, 139.5676..."
235608,50244-12804-i-8185,2022/06/06,,0,2500,2701,,,通常部,市区町村道等,...,,,,22,1696,580,139.566873,35.568595,0540-00,"MULTILINESTRING ((139.56814 35.56855, 139.5676..."
235609,50244-12804-i-8181,2022/06/06,,0,2500,2701,,,通常部,市区町村道等,...,,,,22,1696,580,139.566873,35.568595,0540-00,"MULTILINESTRING ((139.56757 35.57036, 139.5684..."


In [ ]:
point_data_test

,ANGLE,FC,FUKASA,FUTAJIBAN1,FUTAJIBAN2,FUTAJIBAN3,FUTAJIBAN4,FUTAJIBAN5,FUTAKEY1,FUTAKEY2,...,NAIKEIV,PUMP,SISETU,SUISOKU,TAKASA,TYPE,YOTO,ZAISITU,ZANCHI,geometry
0,137.001700,12245,0.0,82.12,0.00,0.00,0.0,0.0,1,None,...,0.0,0,0,0,0.0,2,0,14,0,POINT (35.748 139.333)
1,93.124200,12144,0.0,0.00,0.00,0.00,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT (35.748 139.333)
2,114.999200,12244,0.0,0.00,0.00,0.00,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT (35.748 139.334)
3,62.999300,12144,0.0,0.00,0.00,0.00,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT (35.748 139.334)
4,122.000500,12244,0.0,0.00,0.00,0.00,0.0,0.0,None,None,...,0.0,0,0,0,0.0,1,0,14,0,POINT (35.748 139.334)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615188,203.732706,12344,0.0,38.69,2.12,3.56,0.0,0.0,1,2,...,0.0,9,0,0,0.0,1,31,99,6,POINT (35.849 139.279)
615189,199.249267,12344,0.0,16.86,2.12,3.56,0.0,0.0,1,2,...,0.6,9,0,0,0.0,40,31,99,6,POINT (35.835 139.249)
615190,124.629046,12344,0.0,39.21,2.12,3.56,0.0,0.0,1,2,...,0.0,9,0,0,0.0,4,31,99,6,POINT (35.778 139.258)
615191,77.305967,12344,0.0,3.97,2.12,3.56,0.0,0.0,1,2,...,0.0,9,0,0,0.0,1,31,99,6,POINT (35.822 139.252)


In [ ]:
print(point_data_test.crs)

EPSG:4326


In [ ]:
point_data_test = point_data_test.to_crs(epsg=4326)
road_data_test = road_data_test.to_crs(epsg=4326)

In [ ]:
result = sjoin_nearest(point_data_test, road_data_test, how="left", lsuffix="point", rsuffix="road")

c:\Users\jin12\anaconda3\envs\god\lib\site-packages\geopandas\array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [ ]:
print(result.columns)

Index(['ANGLE', 'FC', 'FUKASA', 'FUTAJIBAN1', 'FUTAJIBAN2', 'FUTAJIBAN3',
       'FUTAJIBAN4', 'FUTAJIBAN5', 'FUTAKEY1', 'FUTAKEY2', 'FUTAKEY3',
       'FUTAKEY4', 'FUTAKEY5', 'FUTASUU', 'G1KANRI', 'G1KYOKA', 'G1KYOKAYMD',
       'G2KANRI', 'G2KYOKA', 'G2KYOKAYMD', 'GAIKEIH', 'GAIKEIV', 'HAIJYO',
       'HIKITUGI', 'JIBANTAKA', 'JINSIN', 'JINX', 'JINY', 'KAIDAN', 'KANEDA',
       'KENSETUYMD', 'KEY', 'KITEN', 'KUMITATE', 'NAIKEIH', 'NAIKEIV', 'PUMP',
       'SISETU', 'SUISOKU', 'TAKASA', 'TYPE', 'YOTO', 'ZAISITU', 'ZANCHI',
       'geometry', 'index_road', 'rID', 'lfSpanFr', 'lfSpanTo', 'tmpFlg',
       'orgGILvl', 'ftCode', 'admCode', 'devDate', 'type', 'rdCtg', 'state',
       'lvOrder', 'name', 'admOfcRd', 'rnkWidth', 'Width', 'sectID',
       'tollSect', 'medSect', 'motorway', 'repLtdLvl', 'rtCode', 'lat', 'lon',
       'fid_2', 'KEY_CODE', 'PREF', 'CITY', 'S_AREA', 'PREF_NAME', 'CITY_NAME',
       'S_NAME', 'KIGO_E', 'HCODE', 'AREA', 'PERIMETER', 'R2KAxx', 'R2KAxx_ID',
       'KIH

In [ ]:
point_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 615193 entries, 0 to 615192
Data columns (total 45 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ANGLE       615193 non-null  float64 
 1   FC          615193 non-null  int64   
 2   FUKASA      615193 non-null  float64 
 3   FUTAJIBAN1  615193 non-null  float64 
 4   FUTAJIBAN2  615193 non-null  float64 
 5   FUTAJIBAN3  615193 non-null  float64 
 6   FUTAJIBAN4  615193 non-null  float64 
 7   FUTAJIBAN5  615193 non-null  float64 
 8   FUTAKEY1    133943 non-null  object  
 9   FUTAKEY2    8019 non-null    object  
 10  FUTAKEY3    4030 non-null    object  
 11  FUTAKEY4    46 non-null      object  
 12  FUTAKEY5    11 non-null      object  
 13  FUTASUU     615193 non-null  int64   
 14  G1KANRI     574383 non-null  object  
 15  G1KYOKA     1 non-null       object  
 16  G1KYOKAYMD  615193 non-null  int64   
 17  G2KANRI     574387 non-null  object  
 18  G2KYOKA     3948

In [ ]:
road_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 235611 entries, 0 to 235610
Data columns (total 56 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   WKT         235611 non-null  object  
 1   rID         235611 non-null  object  
 2   lfSpanFr    235611 non-null  object  
 3   lfSpanTo    235611 non-null  object  
 4   tmpFlg      235611 non-null  object  
 5   orgGILvl    235611 non-null  object  
 6   ftCode      235611 non-null  object  
 7   admCode     235611 non-null  object  
 8   devDate     235611 non-null  object  
 9   type        235611 non-null  object  
 10  rdCtg       235611 non-null  object  
 11  state       235611 non-null  object  
 12  lvOrder     235611 non-null  object  
 13  name        235611 non-null  object  
 14  admOfcRd    235611 non-null  object  
 15  rnkWidth    235611 non-null  object  
 16  Width       235611 non-null  object  
 17  sectID      235611 non-null  object  
 18  tollSect    2356

In [ ]:
result.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 620035 entries, 0 to 615192
Data columns (total 100 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   ANGLE        620035 non-null  float64 
 1   FC           620035 non-null  int64   
 2   FUKASA       620035 non-null  float64 
 3   FUTAJIBAN1   620035 non-null  float64 
 4   FUTAJIBAN2   620035 non-null  float64 
 5   FUTAJIBAN3   620035 non-null  float64 
 6   FUTAJIBAN4   620035 non-null  float64 
 7   FUTAJIBAN5   620035 non-null  float64 
 8   FUTAKEY1     135148 non-null  object  
 9   FUTAKEY2     8074 non-null    object  
 10  FUTAKEY3     4052 non-null    object  
 11  FUTAKEY4     47 non-null      object  
 12  FUTAKEY5     12 non-null      object  
 13  FUTASUU      620035 non-null  int64   
 14  G1KANRI      578820 non-null  object  
 15  G1KYOKA      1 non-null       object  
 16  G1KYOKAYMD   620035 non-null  int64   
 17  G2KANRI      578824 non-null  object  
